# Парсинг

Модели, которые хорошо следуют заданным в промпте инструкциям, могут возвращать данные в заданном формате.

В основе представленного подхода лежит правильно подготовленный промпт и последующий парсинг ответа модели.

In [1]:
from langchain_community.chat_models.gigachat import GigaChat

llm = GigaChat(
    verify_ssl_certs=False,
    timeout=6000,
    model="GigaChat-Pro",
    temperature=0.01,
)

:::note

Подробнее о повышении качества извлечения информации — в разделе [Рекомендации](/docs/use_cases/extraction/guidelines).

В этом разделе представлен простой пример, но для получения наиболее хорошего результата, в промпте модели следует предоставить примеры.

:::

## Использование PydanticOutputParser

Пример ниже демонстрирует разбор ответа модели с помощью встроенного парсера `PydanticOutputParser`.

In [2]:
from typing import List, Optional

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator


class Person(BaseModel):
    """Информация о человеке."""

    name: str = Field(..., description="Имя человека")
    height_in_meters: float = Field(..., description="Высота человека в метрах")


class People(BaseModel):
    """Выдели всю информацию о людях в тексте."""

    people: List[Person]


# Инициализация парсера
parser = PydanticOutputParser(pydantic_object=People)

# Промпт
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Ответь на запрос пользователя в формате JSON. Schema Information: \n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

Запрос, который передается в модель.

In [3]:
query = "Anna is 23 years old and she is 6 feet tall"

In [4]:
print(prompt.format_prompt(query=query).to_string())

System: Ответь на запрос пользователя в формате JSON. Schema Information: 
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Выдели всю информацию о людях в тексте.", "properties": {"people": {"title": "People", "type": "array", "items": {"$ref": "#/definitions/Person"}}}, "required": ["people"], "definitions": {"Person": {"title": "Person", "description": "Информация о человеке.", "type": "object", "properties": {"name": {"title": "Name", "description": "Имя человека", "type": "string"}, "height_in_meters": {"title": "Height In Meters", "description": "Высота человека в метр

In [5]:
chain = prompt | llm | parser
chain.invoke({"query": query})

People(people=[Person(name='Anna', height_in_meters=1.8)])

## Использование собственного парсера

Используя `LangChain` и язык `LCEL` вы можете создать собственный промпт и парсер.

Для разбора ответа модели можно использовать несложную функцию.

In [6]:
import json
import re
from typing import List, Optional

from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator


class Person(BaseModel):
    """Информация о человеке."""

    name: str = Field(..., description="Имя человека")
    height_in_meters: float = Field(..., description="Высота человека в метрах")


class People(BaseModel):
    """Выдели всю информацию о людях в тексте."""

    people: List[Person]


# Инициализация парсера
parser = PydanticOutputParser(pydantic_object=People)

# Промпт
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Ответь на запрос пользователя в формате JSON. Schema Information: \n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())


# Собственный парсер
def extract_json(message: AIMessage) -> List[dict]:
    """Парсер JSON контента"""
    text = message.content

    # Возвращение списка подходящихстрок JSON и удаление всех ведущих и завершающих пробельных символов
    try:
        return json.loads(text)
    except Exception:
        raise ValueError(f"Failed to parse: {message}")

In [7]:
query = "Anna is 23 years old and she is 6 feet tall"
print(prompt.format_prompt(query=query).to_string())

System: Ответь на запрос пользователя в формате JSON. Schema Information: 
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Выдели всю информацию о людях в тексте.", "properties": {"people": {"title": "People", "type": "array", "items": {"$ref": "#/definitions/Person"}}}, "required": ["people"], "definitions": {"Person": {"title": "Person", "description": "Информация о человеке.", "type": "object", "properties": {"name": {"title": "Name", "description": "Имя человека", "type": "string"}, "height_in_meters": {"title": "Height In Meters", "description": "Высота человека в метр

In [8]:
chain = prompt | llm | extract_json
chain.invoke({"query": query})

{'people': [{'name': 'Anna', 'height_in_meters': 1.8}]}